#### 1472. Design Browser History

* https://leetcode.com/problems/design-browser-history/description/

In [1]:
import threading
class BrowserHistory:
    """
        Approach - Array + Pointer
        All operations - O(1)
        Space = O(1) except visit which is O(n)
    """

    __slots__ = ('_history', '_current_index', '_max_index', '_lock')

    def __init__(self, homepage: str):
        self._history = [homepage]
        self._current_index = 0
        self._max_index = 0
        self._lock = threading.Lock()

        
    def visit(self, url: str) -> None:
        with self._lock:
            self._current_index += 1
            if self._current_index < len(self._history):
                self._history[self._current_index] = url
            else:
                self._history.append(url)
            
            self._max_index = self._current_index

        
    def back(self, steps: int) -> str:
        with self._lock:
            self._current_index = max(0, self._current_index - steps)
            return self._history[self._current_index]

        
    def forward(self, steps: int) -> str:
        with self._lock:
            self._current_index = min(self._max_index, self._current_index + steps)
            return self._history[self._current_index]

        


# Your BrowserHistory object will be instantiated and called as such:
# obj = BrowserHistory(homepage)
# obj.visit(url)
# param_2 = obj.back(steps)
# param_3 = obj.forward(steps)

In [ ]:
# Follow up question on memory
#Let’s say this platform is used by thousands of clients simultaneously, and we need to strictly limit memory usage per user session. 
# How would you implement a 'Max History' limit (e.g., 500 pages) using your current array-based approach? What are the trade-offs of using an array 
# vs. a Doubly Linked List in that specific 'capped' scenario?
#To support a max_size in an Enterprise Product context, we need to transition from a simple dynamic array to a Circular Buffer (Ring Buffer) logic.
# If we simply stop appending when we hit max_size, the user can't visit new pages. If we use pop(0) to make room, we hit an $O(N)$ performance penalty. 
# A circular buffer allows us to overwrite the oldest entries in $O(1)$ time.

import threading
class BrowserHistory:
    """
        Approach - Array + Pointer
        All operations - O(1)
        Space = O(1) except visit which is O(n)
    """

    __slots__ = ('_history', '_current_index', '_max_bound', '_lock', '_max_size')

    def __init__(self, homepage: str, max_size: int = 5000):
        self._history = [None]*max_size
        self._history = [homepage]
        self._current_index = 0
        self._max_bound = 0
        self._max_size = max_size
        self._lock = threading.Lock()

        
    def visit(self, url: str) -> None:
        with self._lock:
            self._current_index += 1
            if self._current_index < len(self._history):
                self._history[self._current_index] = url
            else:
                self._history.append(url)
            
            self._max_bound = self._current_index

        
    def back(self, steps: int) -> str:
        with self._lock:
            self._current_index = max(0, self._current_index - steps)
            return self._history[self._current_index]

        
    def forward(self, steps: int) -> str:
        with self._lock:
            self._current_index = min(self._max_bound, self._current_index + steps)
            return self._history[self._current_index]

        


# Your BrowserHistory object will be instantiated and called as such:
# obj = BrowserHistory(homepage)
# obj.visit(url)
# param_2 = obj.back(steps)
# param_3 = obj.forward(steps)

In [77]:
%%timeit

# Perfomance wise optimized solution


class BrowserHistory:

    def __init__(self, homepage: str):
        self.store = [homepage]
        self.curr = 0
        self.max_len = 1 # to be used to maintain the max val in case of new path creation
        

    def visit(self, url: str) -> None:
        self.curr += 1
        if self.curr < len(self.store):
            self.store[self.curr] = url
        else:
            self.store.append(url)
        self.max_len = self.curr + 1


    def back(self, steps: int) -> str:
        self.curr = max(0, self.curr-steps)
        return self.store[self.curr]


    def forward(self, steps: int) -> str:
        self.curr = min(self.max_len-1, self.curr+steps)
        return self.store[self.curr]
    

browserHistory  = BrowserHistory('leetcode.com')
browserHistory.visit('google.com')
browserHistory.visit("facebook.com")
browserHistory.visit("youtube.com")
browserHistory.back(1)
browserHistory.back(1)
browserHistory.forward(1)
browserHistory.visit("linkedin.com")
browserHistory.forward(2)
browserHistory.back(2)
browserHistory.back(7)

8.23 μs ± 785 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [78]:
%%timeit

# # Space wise optimized solution

class BrowserHistory:

    def __init__(self, homepage: str):
        self.store = [homepage]
        self.curr = 0

    def visit(self, url: str) -> None:
        self.store = self.store[:self.curr+1] # truncate the histor
        self.store.append(url)
        self.curr = self.curr + 1

    def back(self, steps: int) -> str:
        self.curr = max(0, self.curr-steps)
        return self.store[self.curr]

    def forward(self, steps: int) -> str:
        self.curr = min(len(self.store)-1, self.curr+steps)
        return self.store[self.curr]
    
browserHistory  = BrowserHistory('leetcode.com')
browserHistory.visit('google.com')
browserHistory.visit("facebook.com")
browserHistory.visit("youtube.com")
browserHistory.back(1)
browserHistory.back(1)
browserHistory.forward(1)
browserHistory.visit("linkedin.com")
browserHistory.forward(2)
browserHistory.back(2)
browserHistory.back(7)

8.33 μs ± 357 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [51]:
# Doubly Linked List solution

class ListNode:
    def __init__(self, val, prev=None, next=None):
        self.val = val
        self.prev = prev
        self.next = next

class BrowserHistory:

    def __init__(self, homepage: str):
        self.curr = ListNode(homepage)

    def visit(self, url: str) -> None:
        self.curr.next = ListNode(url, self.curr)
        self.curr = self.curr.next

    def back(self, steps: int) -> str:
        while self.curr.prev and steps > 0: # steps handles the situation of out of bounds
            self.curr = self.curr.prev
            steps -= 1
        return self.curr.val

    def forward(self, steps: int) -> str:
        while self.curr.next and steps > 0:
            self.curr = self.curr.next
            steps -= 1
        return self.curr.val

'facebook.com'